**Importando bibliotecas**

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

**Abrindo um arquivo CSV do drive**

In [17]:
base = pd.read_csv('../Bases/Criados/base_plano_cartesiano.csv')

In [18]:
base

,V0001,V0024,UPA_PNS,V0006_PNS,C00301,V0026,B001,C006,C008,C009,...,Inicio_tabagismo,Tempo_parou_fumar,Intensidade_fumo,Doses_semanais,Quant_doses_alcool_semanal,horas_dispositivos_tv,consumo_frutas_verduras,consumo_alimentos_nsaudaveis,imc,faixa_imc
0,21,2110011,210003272,2,1.0,1,1.0,1.0,81.0,4.0,...,Início antes dos 18,>5 anos,0.0,0.0,0 doses,8.0,6.0 3.0,0.0 2.0 1.0 3.0,25.88,Sobrepeso
1,21,2110011,210003272,4,1.0,1,2.0,2.0,54.0,4.0,...,Não fuma,Nunca fumou,0.0,0.0,0 doses,7.0,2.0 5.0,0.0 1.0 0.0 0.0,34.05,Obesidade Grau I
2,21,2110011,210003272,5,1.0,1,2.0,2.0,41.0,2.0,...,Não fuma,>5 anos,0.0,15.0,15–30 doses,4.0,1.0 3.0,0.0 4.0 0.0 3.0,36.89,Obesidade Grau II
3,21,2110011,210003272,7,1.0,1,2.0,1.0,62.0,1.0,...,Início antes dos 18,>5 anos,0.0,8.0,6–14 doses,6.0,3.0 7.0,0.0 0.0 0.0 0.0,28.38,Sobrepeso
4,21,2110011,210003272,8,2.0,1,2.0,2.0,51.0,4.0,...,Não fuma,Nunca fumou,0.0,0.0,0 doses,7.0,6.0 6.0,0.0 0.0 0.0 0.0,28.96,Sobrepeso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24313,53,5310220,530044434,5,1.0,2,2.0,2.0,51.0,1.0,...,Não fuma,Nunca fumou,0.0,7.0,6–14 doses,4.0,5.0 7.0,0.0 0.0 0.0 0.0,24.84,Peso normal
24314,53,5310220,530044434,6,2.0,2,2.0,1.0,44.0,2.0,...,Não fuma,Nunca fumou,0.0,0.0,0 doses,8.0,3.0 2.0,2.0 1.0 7.0 0.0,23.62,Peso normal
24315,53,5310220,530044434,9,2.0,2,2.0,2.0,54.0,4.0,...,Início entre 18 e 30,Fumante atual,7.0,3.0,1–5 doses,4.0,7.0 2.0,0.0 1.0 0.0 0.0,22.32,Peso normal
24316,53,5310220,530044434,10,2.0,2,3.0,1.0,44.0,4.0,...,Início após os 30,Fumante atual,7.0,24.0,15–30 doses,10.0,7.0 3.0,0.0 0.0 0.0 0.0,25.51,Sobrepeso


**Verificando ausência na base de dados**

In [19]:
# índices das linhas que contém valores NaN
missing_indices = base[base.isnull().any(axis=1)].index

# imprime apenas as linhas com valoes ausentes
display(base.iloc[missing_indices])

,V0001,V0024,UPA_PNS,V0006_PNS,C00301,V0026,B001,C006,C008,C009,...,Inicio_tabagismo,Tempo_parou_fumar,Intensidade_fumo,Doses_semanais,Quant_doses_alcool_semanal,horas_dispositivos_tv,consumo_frutas_verduras,consumo_alimentos_nsaudaveis,imc,faixa_imc


**Verificar se existem amostras inconsistentes e redundantes**

In [20]:
Nome_das_Colunas = base.columns[:-1]

In [21]:
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]
# se houver valores redundantes ou inconsistentes, imprima
if len(df_duplicates)>0:
    print('\nAmostras redundantes ou inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem valores duplicados')

Não existem valores duplicados


Como podemos ver, existem algumas amostras redundantes (duplicadas) e outras inconsistentes (amostras iguais, mas com classes distintas).
Primeiro, serão removidas as amostras redundantes, mantendo na base apenas uma delas.

In [22]:
def delDuplicatas( df_dataset ):
    '''
    Para cada grupo de amostras duplicadas, mantém uma e apaga as demais
    '''

    # remove as amostras duplicadas, mantendo apenas a primeira ocorrencia
    df_dataset = df_dataset.drop_duplicates(keep = 'first')

    return df_dataset

base = delDuplicatas( base )

**Após remover as amostras redundantes, é preciso checar se há amostras inconsistentes.**

In [23]:
# para detectar inconsistências, a rotina abaixo obtém as amostras onde os valores
# dos atributos continuam duplicados. Neste caso, os atributos serão iguais, mas as classes serão distintas
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]

# se tiver valores inconsistentes, imprime
if len(df_duplicates)>0:
    print('\nAmostras inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem mostras inconsistentes')


Não existem mostras inconsistentes


**Tendo amostras inconsistentes, será necessário apagar estas instâncias, já que não sabemos qual delas está correta**

In [24]:
def delInconsistencias(df_dataset):
    '''
    Remove todas as amostras inconsistentes da base de dados
    '''

    df_dataset = df_dataset.drop_duplicates(subset=Nome_das_Colunas,keep=False)

    return df_dataset

base = delInconsistencias(base)

# obtém apenas as amostras onde os valores dos atributos estão duplicados
df_duplicates = base[base.duplicated(subset=Nome_das_Colunas,keep=False)]

# se tiver valores redundantes ou inconsistentes, imprime
if len(df_duplicates)>0:
    display(df_duplicates)
else:
    print('Não existem amostras redundantes ou inconsistentes')


Não existem amostras redundantes ou inconsistentes
